# 🦌 Make the ELK App Interactive 🕹️

This notebook shows how you can make the ELK App work dynamically with various types of
graphs

In [ ]:
if __name__ == "__main__":
    %pip install -q -r requirements.txt

In [ ]:
import random
from collections import defaultdict
from pprint import pprint

import importnb
import ipywidgets as W
import networkx as nx
from IPython.display import JSON, display

import ipyelk

In [ ]:
with importnb.Notebook():
    from __03_App import a_styled_elk_app_example

In [ ]:
def make_random_forest(number_of_nodes, hierarchy_roots=1, seed=None):
    """Create a random directed graph that meets NetworkX's forest criteria"""
    if seed is not None:
        random.seed(seed)

    if hierarchy_roots < 1:
        return None

    unassigned = set(range(number_of_nodes))
    assigned = set(random.sample(list(unassigned), hierarchy_roots))
    unassigned -= assigned

    tree_edges = []
    while unassigned:
        node = random.sample(list(unassigned), 1)[0]
        tree_edges.append((random.sample(list(assigned), 1)[0], node))
        unassigned -= {node}
        assigned |= {node}

    return nx.DiGraph(tree_edges)

In [ ]:
from ipyelk.elements import iter_edges

In [ ]:
def an_elk_in_a_random_forest():
    box, elk = a_styled_elk_app_example()
    box.layout.flex = "1"
    box.layout.height = "100%"
    out = W.Output()
    json_out = W.Output()
    loader = ipyelk.NXLoader(root_id="root")

    def _update_json(change):
        json_out.outputs = []
        json_out.append_display_data(JSON(elk.view.source.value.dict()))

    elk.view.source.observe(_update_json, "value")

    with out:

        @W.interact
        def make_graph(
            number_of_nodes=(5, 20),
            percent_of_edges=(1, 99),
            hierarchy_roots=(0, 5),
            seed=(0, 1024),
            debug=False,
            padding=(0, 100),
        ):
            hierarchy = make_random_forest(
                number_of_nodes=number_of_nodes,
                hierarchy_roots=hierarchy_roots,
                seed=seed,
            )

            number_of_edges = max(1, int(number_of_nodes * 0.01 * percent_of_edges))

            graph = nx.generators.random_graphs.barabasi_albert_graph(
                n=number_of_nodes,
                m=number_of_edges,
                seed=seed,
            )

            for edge in sorted(graph.edges):
                graph.edges[edge]["sourcePort"] = str(edge)
                graph.edges[edge]["targetPort"] = str(edge)

            elk.source = loader.load(graph=graph, hierarchy=hierarchy)

            if debug:
                counter = defaultdict(list)
                [counter[src].append(tgt) for src, tgt in graph.edges]
                pprint(dict(counter))
                pprint(len(list(iter_edges(elk.view.source.value))))

    return (
        W.HBox(
            [W.VBox([make_graph.widget, json_out]), box],
            layout=dict(height="100%", flex="1"),
        ),
        elk,
    )

In [ ]:
if __name__ == "__main__":
    box, elk = an_elk_in_a_random_forest()
    display(box)
    # display pipe status widget for details
    display(elk.pipe)

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).